<a href="https://colab.research.google.com/github/DonggiChae/practice/blob/master/mask_r_cnn_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torchvision
import torch
import numpy as np
import torch.nn as nn
from torchvision.ops.boxes import batched_nms, box_iou, clip_boxes_to_image, remove_small_boxes
import torch.nn.functional as F
import torchvision.transforms as T
import torch.nn.init as nt
from torchvision.ops import roi_align

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# trf = T.compose([T.resize(),T.ToTensor()])
# input_img = trf(img).unsqueeze(0)

image = torch.zeros((1, 3, 800, 800)).float() 

bbox = torch.FloatTensor([[20, 30, 400, 500], [300, 400, 500, 600]]) # dummy box 
labels = torch.LongTensor([6, 8]) 

fe_size = 50


dummy_img = torch.zeros((1, 3, 800,800)).float()
# print(dummy_img)
#Out: torch.Size([1, 3, 800, 800])


model = torchvision.models.vgg16(pretrained=True)



def feature_extractor(model, image,fe_size):
    fe = list(model.features)
    req_features = []
    k = image.clone()
    for i in fe:
        k = i(k)
        # print(k.size())
        if k.size()[2] < fe_size:
            break
        req_features.append(i)
    faster_rcnn_fe_extractor = nn.Sequential(*req_features)
    out_featuremap = faster_rcnn_fe_extractor(image)
    return out_featuremap

out_map = feature_extractor(model, dummy_img,fe_size)




In [ ]:
# #Anchor Generator
anchor_size = [128, 256, 512]
anchor_ratio = [0.5, 1, 2]
anchors = len(anchor_size) * len(anchor_ratio) * out_map.size(2) * out_map.size(3)
h_ratios = np.sqrt(anchor_ratio)
w_ratios = 1 / h_ratios
anchor_boxes = torch.zeros((len(anchor_ratio) * len(anchor_size) * 50 * 50, 4) )


def ctr_point(ctr_x, ctr_y):
    index = 0
    for x in range(len(ctr_x)):
        for  y in range(len(ctr_y)):
            ctr[index, 0] = ctr_x[x] - 8
            ctr[index, 1] = ctr_y[y] - 8
            index += 1
    return ctr



def anchor_boxes_each_point(anchor_boxes, ctr_x, ctr_y, anchor_size, anchor_ratio):
   index = 0
   for c in ctr:
        ctr_y, ctr_x = c
        # print(c)
        for i in range(len(anchor_size)):
            for j in range(len(anchor_ratio)):
                h_ratios = np.sqrt(anchor_ratio[j])
                w_ratios = 1 / h_ratios
                h = anchor_size[i] * h_ratios
                w = anchor_size[i] * w_ratios
                anchor_boxes[index, 0] =  ctr_x - w / 2
                anchor_boxes[index, 1] =  ctr_y - h / 2
                anchor_boxes[index, 2] =  ctr_x + w / 2
                anchor_boxes[index, 3] =  ctr_y + h / 2
                index += 1
            return anchor_boxes

ctr_x = np.arange(16, (out_map.size(2)+1) * 16, 16)
ctr_y = np.arange(16, (out_map.size(2)+1) * 16, 16)

ctr = np.zeros((2500, 2))


ctr_point(ctr_x, ctr_y)
anchor_boxes_each_point(anchor_boxes, ctr_x, ctr_y, anchor_size, anchor_ratio)


print(anchor_boxes.shape)
print(bbox.shape)






In [ ]:
#iou
pos_iou_threshold = 0.7
neg_iou_threshold = 0.3


ious = box_iou(anchor_boxes, bbox)


gt_argmax_ious = ious.argmax(axis=0)
# print(gt_argmax_ious)

gt_max_ious = ious[gt_argmax_ious, np.arange(ious.shape[1])]
# print(gt_max_ious)

gt_argmax_ious = np.where(ious == gt_max_ious)[0]
# print(gt_argmax_ious)
# print(np.arange(ious.shape[1]))


# label = np.empty(22500, )
# label.fill(-1)
# label[gt_argmax_ious] = 1
# print(gt_max_ious)
# print(pos_iou_threshold)
# print(label.shape)



# for index,i in enumerate(label):
#     if gt_max_ious[index] >= pos_iou_threshold:
#         label[index] = 1
    
# # label = np.where(gt_max_ious >= pos_iou_threshold)
# # label[gt_max_ious < neg_iou_threshold] = 0





In [ ]:
#Determine vaild anchor, positve = 1, negative = 0 , (0.7 > i don't need > 0.3)
pos_ratio = 0.5
randomly_sample = 256
pos = randomly_sample * pos_ratio 








In [ ]:
#bbox regression 
#predicting bounding box
# t_{x} = (x - x_{a})/w_{a}
# t_{y} = (y - y_{a})/h_{a}
# t_{w} = log(w/ w_a)
# t_{h} = log(h/ h_a)
#groung truth box associate with a positive anchor
# t_{x*} = (x* - x_{a})/w_{a}
# t_{y*} = (y* - y_{a})/h_{a}
# t_{w*} = log(w*/ w_a)
# t_{h*} = log(h*/ h_a)









In [ ]:
#rpn_head
#num_anchors = predicted box
layer_weight = 0.1
layer_bias = 0

in_channels = 512
num_anchors = len(anchor_size) * len(anchor_ratio)

conv = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1)
box_layer = nn.Conv2d(in_channels, num_anchors * 4, kernel_size=1, stride=1)
score_layer = nn.Conv2d(in_channels, num_anchors, kernel_size=1, stride=1)
conv.weight.data.normal_(layer_weight, std=0.01)
box_layer.weight.data.normal_(layer_weight, std=0.01)
score_layer.weight.data.normal_(layer_weight, std=0.01)
conv.bias.data.zero_()
box_layer.bias.data.zero_()
score_layer.bias.data.zero_()


x = conv(out_map)
t = F.relu(conv(x))
pred_anchor_locs = box_layer(t)
pred_cls_scores = score_layer(t)
print(pred_cls_scores.shape)


pred_anchor_locs = pred_anchor_locs.permute(0, 2, 3, 1).contiguous().view(1, -1, 4)
pred_cls_scores = pred_cls_scores.permute(0, 2, 3, 1).contiguous()
objectness_score = pred_cls_scores.view(1, 50, 50, 9).contiguous().view(1, -1)
pred_cls_scores  = pred_cls_scores.view(1, -1, 2)


print(pred_anchor_locs.shape)
print(pred_cls_scores.shape)
print(objectness_score.shape)
print(pred_cls_scores.shape)



In [ ]:
#rpn loss 


rpn_loc = pred_anchor_locs[0]
rpn_score = pred_cls_scores[0]
# gt_rpn_loc = 
# gt_rpn_score = 

def smooth_l1_loss(predicted_bbox, bbox_target):
    n= torch.abs(predicted_bbox - bbox_target)
    if n < 1:
        loss = 0.5 * n ** 2
    else:
        loss = n - 0.5
    return loss


rpn_cls_loss = F.cross_entropy(rpn_score,anchor_labels)
rpn_loc_loss = smooth_l1_loss(predicted_bbox, bbox_target).sum()

#rpn loss function
rpn_lambda = 10.
N_reg = (gt_rpn_score >0).float().sum() # number of bounding box for positve anchors
rpn_loc_loss = rpn_loc_loss / N_reg
rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss)

In [ ]:
#nms

nms_thresh = 0.7




In [ ]:
#roi_align


roi = roi_align(input, boxes, output_size = 7, spatial_scale)




#Loss function

reg_loss = 0

def smooth_l1_loss(predicted_bbox, bbox_target):
    n= torch.abs(predicted_bbox - bbox_target)
    if n < 1:
        loss = 0.5 * n ** 2
    else:
        loss = n - 0.5
    return loss


classification_loss = F.cross_entropy(class_score, labels)


rpn_cls_loss = classification_loss
rpn_loc_loss = smooth_l1_loss(predicted_bbox, bbox_target).sum()

#rpn loss function
rpn_lambda = 10.
N_reg = (gt_rpn_score >0).float().sum() # number of bounding box for positve anchors
rpn_loc_loss = rpn_loc_loss / N_reg
rpn_loss = rpn_cls_loss + (rpn_lambda * rpn_loc_loss)

 
#  x = #pixel 
# mask_binary_loss = sigmoid()

